<a href="https://colab.research.google.com/github/PouriaRouzrokh/AI-Deep-Learning-Lab-2022/blob/PR-nb-01/sessions/yolo/YOLO_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Part 0: Setting up the working directory
We start our work by installing some required libraries (including YOLOv5) and cloning our DICOMs and ground truths from a GitHub repository that we have made before. In this notebook, we will train our model on 3500 DICOMs from the CQ500 dataset. Feel free to download the entire dataset and train stronger models on it after this workshop!

In [1]:
import shutil
# Building the folders needed for data splitting
# These folders will be added to our working directory.
import os 

valid_imgs_dir = 'model_data_cropped_SP/images/valid'
valid_labels_dir = 'model_data_cropped_SP/labels/valid'


os.makedirs(valid_imgs_dir, exist_ok=True)
os.makedirs(valid_labels_dir, exist_ok=True)



Here we create three lists based on our dataframe: 

*   X: A list of all SOPInstanceUIDs we got from all rows (don't forget that each DICOM has a unique SOPInstanceUID, so this list includes all our DICOM names).
*   Y: A list of all labels (or the types of hemorrhage lesions) for all rows.
*   groups: A list of all StudyInstanceUIDs.

What we need to do, is to split our X into 2 different folds (80% training and 20% validation), while controlling for the group variable (so that all rows for one specific StudyInstaceUID tag go to either training or validation sets) and also stratifying our data based on the labels (so that the distribution of different labels between training and validation sets be as close as possible to our 80-20 split).

In [2]:
# Splitting the data into training and validation sets.
#from sklearn.model_selection import StratifiedGroupKFold
from sklearn.model_selection import train_test_split
import pandas as pd 

csv_dir = '/media/xintie/DATA/siim-covid19-detection/csv/'
labels_df = pd.read_csv(csv_dir+'train_lung_bbox_SP.csv')

# To make the cell work prperly if run multiple times.
val_df = labels_df


/home/xintie/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [3]:
# Moving the images to their folders in the "model_data" directory
import shutil
png_type = 'SP_png_cropped'

image_dir = '/media/xintie/DATA/siim-covid19-detection/'
image_folder = image_dir + '{}/train/'.format(png_type)

YOLO needs us two TXT files including the paths to images in our training and validation directories as well, so let's create those files:

In [6]:
# To make the cell work prperly if run multiple times. 
shutil.rmtree('model_data_cropped_SP/val.txt', ignore_errors=True)

# Building the txt files including paths to all images in training and validation sets
        
with open('model_data_cropped_SP/val.txt', 'w') as f:
    for file in os.listdir(valid_imgs_dir):
        f.write(os.path.join(valid_imgs_dir, file)+'\n')
        

OK, now we should create the labels for training our YOLO model.
YOLO needs a TXT file for each image that contains the locations of each bounding box in that image, plus the index for the class of that box in separate lines. 

We will create these files below. For doing this, we first give a unique index to each class of brain hemorrhage in our dataset and then write their location in files. Here is where we will need your coding again:

YOLO needs four coordinates for each box:

*  x_min: The x-coordinate for the upper-left corner of the box in pixels.
*  y_min: The y-coordinate for the upper-left corner of the box in pixels.
*  width: The wideness of the box in pixels.
*  height: The height of the box in pixels.

We should note that the way YOLO receives ground truth labels from us (and further predicts labels) is special. YOLO needs all the above numbers proportional to the size of the image, (and not in their absolute values). For example, if the image size is 512 * 512 (as in our case), and the x-min for an imaginary box is 100, YOLO expects a float number (100/512) as the x-min in our labels. This rule also applies to all other bounding box numbers YOLO needs.

In [7]:
# To make the cell work prperly if run multiple times. 
shutil.rmtree(valid_labels_dir, ignore_errors=True)
os.makedirs(valid_labels_dir, exist_ok=True)

# Creating the labels

label_to_index_dict = {
    'COVID': 0
}

### Part 4: Configuring the YOLOv5 Model
Alright, now that we have all the images and labels set up, we can configure the YOLOv5 model for our training. For doing so, we need to change the YAML file in the model directory. We should give it the path to our training and validation directories of images, the number of classes, and the name of classes.

In [8]:
# Configuring the data.yaml file

import yaml
model_dir = 'yolov5'

data = dict(
    train = '../model_data_cropped_SP/images/train',
    val = '../model_data_cropped_SP/images/valid',   
    nc    = 1, # number of classes
    names = list(label_to_index_dict.keys()) # classes
    )

with open(f'{model_dir}/data/data.yaml', 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

with open(f'{model_dir}/data/data.yaml', 'r') as file:
    for line in file.readlines():
        print(line.strip())

names:
- COVID
nc: 1
train: ../model_data_cropped_SP/images/train
val: ../model_data_cropped_SP/images/valid


### Part 5: View and modify the hyperparameters (optional)
The Ultralytics implementation of YOLO gives us the ability to change many of the hyperparameters YOLO works with. These are all accessible in a YAML file in the model's directory. We will not touch these settings for now, but let's visualize them before we proceed:

In [9]:
with open(f'{model_dir}/data/hyps/hyp.scratch-high.yaml', 'r') as file:
    for line in file.readlines():
        print(line.strip())

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license
# Hyperparameters for high-augmentation COCO training from scratch
# python train.py --batch 32 --cfg yolov5m6.yaml --weights '' --data coco.yaml --img 1280 --epochs 300
# See tutorials for hyperparameter evolution https://github.com/ultralytics/yolov5#tutorials

lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.1  # final OneCycleLR learning rate (lr0 * lrf)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay 5e-4
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.3  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 0.7  # obj loss gain (scale with pixels)
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
fl_gamma: 0.0  # foc

### Part 6: Training
Perfect, now everything is set for us to start the training! Fortunately, the training itself could be run using one line of code! We just need to determine the image size, batch size, number of epochs, the directory to our model's directory, a name for our project, and a name for our current run of experiment. 

Please note that the training command should be run from the command line, that is why we have put an "!" mark before the line we do the training.

**Note: In Google Colab, we cannot run the training with a batch size greater than 8, otherwise we will hit the memory limits. Training with such a small batch size on the other hand may take a lot of time, so we only train our model for one epoch. Please run this notebook locally or give it more time later for a full training.**

# Inference stage 

As the training will probably take longer than the time we have available in this workshop, we will work with a pre-trained model from now on. We have already trained this model using almost the same configuration we set earlier.You can download that model and replace it in a usual place in our working directory by running the following cell:

All right, now we assume that we have done a full training. The Ultralytics implementation of YOLOv5 plots a lot of useful curves and logs much information during the training. You could easily visualize that inforation by looking at your model_dir/project/run directory or even using loggers like TensorBoard or WandB.

**Please note that the best mAP of our model has been about 0.4. Though not very bad, this mAP is not very high. Rather than the training hyperparameters, What other factors do you think can explain this phenomenon?**

### Part 7: Inference
As the final part of our notebook, let's apply our YOLO model to a set of images. For doing this, we separate a part of images that our model had not seen during the training (Please note that these images are different than the validation images you created above, as we are now working with a model that we had trained before; not a model that you trained here). 

In [17]:
# We should now run the detect.py code from the model directory and pass all 
# the arguments that YOLO needs. 
RUN = 'Exp_cropped_medium_pre'

# Setting up the pipeline for inference
PROJECT_dir = f'{model_dir}/COVID_pneumonia_YOLO_Project'
weights_path = f'{PROJECT_dir}/{RUN}/weights/best.pt'
destination_dir = f'{PROJECT_dir}/{RUN}/Inference_Results'
output_name = 'results_SP'

img_size = 512
conf: float = 0.25
iou_threshold: float = 0.2
max_dt = 5
# We should now run the detect.py code from the model directory and pass all 
# the arguments that YOLO needs. 
inference_imgs_dir =  './model_data_cropped_SP/images/valid'

!python {model_dir}/detect.py  --weights {weights_path} \
                               --source {inference_imgs_dir} \
                               --project {destination_dir} \
                               --name {output_name}\
                               --img {img_size} \
                               --conf {conf} \
                               --iou-thres {iou_threshold} \
                               --max-det {max_dt} \
                               --save-txt \
                               --save-conf \
                               --exist-ok


/home/xintie/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
detect: weights=['yolov5/COVID_pneumonia_YOLO_Project/Exp_cropped_medium_pre/weights/best.pt'], source=./model_data_cropped_SP/images/valid, data=yolov5/data/coco128.yaml, imgsz=[512, 512], conf_thres=0.25, iou_thres=0.2, max_det=5, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/COVID_pneumonia_YOLO_Project/Exp_cropped_medium_pre/Inference_Results, name=results_SP, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-30-g342fe05 Python-3.8.5 torch-1.13.0+cu117 CUDA:0 (NVIDIA TITAN RTX, 24220MiB)

Fusing layers... 
Model summary: 212 l

image 38/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03057_ses-E06798_run-1_bp-chest_vp-ap_cr-P-05.png: 512x512 1 COVID, 14.6ms
image 39/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03057_ses-E06851_run-1_bp-chest_vp-ap_cr-P+01.png: 512x512 2 COVIDs, 14.5ms
image 40/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03058_ses-E06152_run-1_bp-chest_vp-ap_dx-P+05.png: 512x512 (no detections), 16.5ms
image 41/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03059_ses-E06498_run-1_bp-chest_vp-ap_cr-P+01.png: 512x512 2 COVIDs, 16.1ms
image 42/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03059_ses-E06720_run-1_bp-chest_vp-ap_cr-P-03.png: 512x512 2 COVIDs, 14.9ms
image 43/12308 /media/xi

image 80/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03071_ses-E06736_run-1_bp-chest_vp-ap_cr-P+05.png: 512x512 1 COVID, 15.8ms
image 81/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03071_ses-E06832_run-1_bp-chest_vp-ap_cr-P+10.png: 512x512 1 COVID, 16.1ms
image 82/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03071_ses-E06872_acq-2_run-1_bp-chest_vp-ap_cr-P+06.png: 512x512 2 COVIDs, 13.9ms
image 83/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03072_ses-E06166_run-1_bp-chest_vp-ap_cr-P+02.png: 512x512 1 COVID, 15.8ms
image 84/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03072_ses-E06229_run-1_bp-chest_vp-pa_dx-P+00.png: 512x512 (no detections), 14.6ms
image 85/12308 /medi

image 123/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03081_ses-E06519_run-1_bp-chest_vp-ap_cr-P+04.png: 512x512 2 COVIDs, 15.7ms
image 124/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03081_ses-E06613_run-1_bp-chest_vp-ap_cr-P+07.png: 512x512 1 COVID, 15.2ms
image 125/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03081_ses-E06751_run-1_bp-chest_vp-ap_cr-P+00.png: 512x512 2 COVIDs, 27.3ms
image 126/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03081_ses-E06882_run-1_bp-chest_vp-ap_cr-P+01.png: 512x512 2 COVIDs, 16.1ms
image 127/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03082_ses-E06176_run-1_bp-chest_vp-ap_dx-P+09.png: 512x512 2 COVIDs, 15.5ms
image 128/12308 /media/xin

image 167/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03093_ses-E06187_run-1_bp-chest_vp-ap_dx-P+08.png: 512x512 2 COVIDs, 15.1ms
image 168/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03093_ses-E06187_run-2_bp-chest_vp-ap_dx-P+08.png: 512x512 2 COVIDs, 15.8ms
image 169/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03094_ses-E06188_run-1_bp-chest_vp-ap_cr-P+04.png: 512x512 2 COVIDs, 15.7ms
image 170/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03094_ses-E06239_run-1_bp-chest_vp-ap_cr-P+02.png: 512x512 2 COVIDs, 15.3ms
image 171/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S03094_ses-E06625_run-1_bp-chest_vp-ap_cr-P+00.png: 512x512 2 COVIDs, 13.7ms
image 172/12308 /media/xi

...

image 12142/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29169_ses-E60426_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 (no detections), 17.0ms
image 12143/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29173_ses-E60432_run-1_bp-chest_vp-pa_dx-N+99.png: 512x512 (no detections), 17.6ms
image 12144/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29174_ses-E60433_run-1_bp-chest_vp-pa_dx-N+99.png: 512x512 (no detections), 18.5ms
image 12145/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29176_ses-E60436_run-1_bp-chest_vp-ap_cr-N+99.png: 512x512 1 COVID, 19.1ms
image 12146/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29178_ses-E60438_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 (no detectio

image 12183/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29293_ses-E60589_run-1_bp-chest_vp-ap_dx-N+99.png: 512x512 1 COVID, 18.2ms
image 12184/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29297_ses-E60595_run-1_bp-chest_vp-ap_dx-N+99.png: 512x512 (no detections), 16.9ms
image 12185/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29299_ses-E60600_run-1_bp-chest_vp-pa_dx-N+99.png: 512x512 (no detections), 15.1ms
image 12186/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29300_ses-E60601_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 1 COVID, 16.4ms
image 12187/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29303_ses-E60607_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 (no detections), 16.

image 12225/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29399_ses-E60742_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 (no detections), 16.5ms
image 12226/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29401_ses-E60745_run-1_bp-chest_vp-ap_dx-N+99.png: 512x512 (no detections), 16.9ms
image 12227/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29403_ses-E60749_run-1_bp-chest_vp-pa_dx-N+99.png: 512x512 (no detections), 16.6ms
image 12228/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29404_ses-E60750_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 (no detections), 14.6ms
image 12229/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29406_ses-E60752_run-1_bp-chest_vp-ap_dx-N+99.png: 512x512 (no 

image 12267/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29523_ses-E60908_run-1_bp-chest_vp-ap_dx-N+99.png: 512x512 2 COVIDs, 17.0ms
image 12268/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29524_ses-E60909_run-1_bp-chest_vp-ap_dx-N+99.png: 512x512 2 COVIDs, 17.1ms
image 12269/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29525_ses-E60910_run-1_bp-chest_vp-ap_cr-N+99.png: 512x512 2 COVIDs, 18.4ms
image 12270/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29527_ses-E60912_run-1_bp-chest_vp-pa_cr-N+99.png: 512x512 (no detections), 16.6ms
image 12271/12308 /media/xintie/DATA/siim-covid19-detection/yolo_pneumonia_detection/model_data_cropped_SP/images/valid/S29533_ses-E60919_run-1_bp-chest_vp-ap_cr-N+99.png: 512x512 2 COVIDs, 17.2ms
image 12

Results saved to yolov5/COVID_pneumonia_YOLO_Project/Exp_cropped_medium_pre/Inference_Results/results_SP
7262 labels saved to yolov5/COVID_pneumonia_YOLO_Project/Exp_cropped_medium_pre/Inference_Results/results_SP/labels


Before we wrap up this notebook, we can also visualize some of the YOLO predictions!

That's it! Congratulations on training and applying your first YOLOv5 model. Hopefully, you have learned how to train more YOLO models on your custom datasets. In case you are interested, there are a few more things you can try with YOLOv5:

*   Changing the hyperparameters (e.g., IOU, learning rate, etc.) and retraining your model.
*   Applying test time augmentation (TTA) during your inference.
*   Exploring the different augmentations YOLO does during the training, especially the mosaic augmentation. 



**Thank you for attending this workshop!**

In [35]:
def Get_GT_from_txt(txtpath):
    with open(txtpath) as f:
        lines = f.readlines()
        array = []
        for line in lines: 
            array.append([x for x in line.split()])
        return array

In [36]:
from tqdm import * 
#destination_dir = f'{PROJECT_dir}/{RUN}/Inference_Results'
label_folder = destination_dir + '/results_SP/labels/'
df = pd.read_csv('test_Swin_SP_roc.csv') 
filename_list = df.Filename.tolist() 
num_boxes = []

for filename in tqdm(filename_list):
    id_name = filename[filename.find('Covid')+6:-4]
    txt_path = label_folder + id_name + '.txt'
    if os.path.exists(txt_path):
        bboxes = Get_GT_from_txt(txt_path)
        num_boxes.append(len(bboxes))
    else:
        num_boxes.append(0)


100%|██████████| 6520/6520 [00:00<00:00, 36938.50it/s]


In [37]:
df['num_boxes'] = num_boxes

In [38]:
df.to_csv('test_Swin_SP_roc_num_boxes.csv',index=False)